In [ ]:

#Librerias
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import os
import seaborn as sns
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten,  Dropout, BatchNormalization, LeakyReLU,Input
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from kaggle_datasets import KaggleDatasets
from keras.models import Model
from tensorflow import keras
!pip install -q efficientnet
import efficientnet.tfkeras as efn


# TPU Configuration


In [ ]:


# TPU or GPU detection
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
    
def seed_everything(seed=0):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 2048
seed_everything(seed)
print("REPLICAS: ", strategy.num_replicas_in_sync)

# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path()
AUTO = tf.data.experimental.AUTOTUNE


In [ ]:
tf.tpu.experimental.initialize_tpu_system(tpu) # Clear TPU Memory

In [ ]:
# Parametros
EPOCAS = 40
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
IMG_SIZE = 700


In [ ]:
#Datos de entrenamiento y test
train = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/train.csv')
test = pd.read_csv('/kaggle/input/plant-pathology-2020-fgvc7/test.csv')


In [ ]:
#-------------------------------PREPARACION DE LA DATA----------------------------------------

#definicion de la funcion de ruta de las imagenes
def format_path(st):
    return GCS_DS_PATH + '/images/' + st + '.jpg'

train_paths = train.image_id.apply(format_path).values
test_paths = test.image_id.apply(format_path).values
train_labels = train.loc[:, 'healthy':].values
SPLIT_VALIDATION =True
if SPLIT_VALIDATION:
    train_paths, valid_paths, train_labels, valid_labels =train_test_split(train_paths, train_labels, test_size=0.15, 
                                                                           random_state=seed)

#funcion para decodificar
def decode_image(filename, label=None, IMG_SIZE=(IMG_SIZE, IMG_SIZE)):

    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, IMG_SIZE)
    
    if label is None:
        return image
    else:
        return image, label
    
#funcion de aumento de la data
def data_augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if label is None:
        return image
    else:
        return image, label



In [ ]:
train_dataset = (
tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .cache()
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

train_dataset_1 = (
tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .cache()
    .map(data_augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(512)
    .batch(64)
    .prefetch(AUTO)
)


valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((valid_paths, valid_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(test_paths)
    .map(decode_image, num_parallel_calls=AUTO)
    .map(data_augment, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
)

    


In [ ]:
#print(list(test_dataset))
#print(list(valid_dataset))

In [ ]:
#-------------------------------CONFIGURACION DEL MODELO----------------------------------------



#Definicion de la tasa de aprendizaje
Tasa_inicio = 0.0001
Tasa_max = 0.00005 * strategy.num_replicas_in_sync
Tasa_min = 0.0001
Etapa2 = 4
Etapa3 = 6
Tasa_decaexpo = .8

def Tasa_aprend(epoca):
    if epoca < Etapa2:
        lr = (Tasa_max - Tasa_inicio) / Etapa2 * epoca + Tasa_inicio
    elif epoca < Etapa2 + Etapa3:
        lr = Tasa_max
    else:
        lr = (Tasa_max - Tasa_min) * Tasa_decaexpo**(epoca - Etapa2 - Etapa3) + Tasa_min
    return lr
    
TA = tf.keras.callbacks.LearningRateScheduler(Tasa_aprend, verbose=True)

#Grafica de la tasa de aprendizaje
rng = [i for i in range(EPOCAS)]
y = [Tasa_aprend(x) for x in rng]
plt.plot(rng, y)
print("La Tasa de aprendizaje esta entre : {:.3g} to {:.3g} ".format(y[0], max(y)))



In [ ]:


#-------------------------------CONFIGURACION DEL MODELO----------------------------------------

with strategy.scope():    
    
    #Arquitectura seleccionada
        #image_shape : forma de la imagen , 700x700x3
        #weights : Pesos iniciales , 
        #con el parametro imagenet ,se utiliza un modelo preentrenado en la base de datos imagennet
        #include_top: incluir o no la capa de conexion completa al final de la red
        
    efficient_net = efn.EfficientNetB7(
                    input_shape=(IMG_SIZE, IMG_SIZE, 3),
                    weights='imagenet',
                    include_top=False
                    )
    x = efficient_net.output
    
    #Capas de agrupamiento o polling
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    #unidades: Entero positivo, dimensionalidad del espacio de salida.
    #activation = funcion de activacion,Relu ,softmax 
    x = tf.keras.layers.Dense(4, activation='softmax')(x)
    
    model =  keras.Model(inputs = efficient_net.input,outputs=x)
    
    #funciones de pérdida : funcion de pérdida de entropía cruzada "crossentropy"
    #La funcion se utiliza durante el entrenamiento para medir el desempeño de la red
    #optimizador adam 
    
    #Compilacion de un modelo en keras
    model.compile(loss="categorical_crossentropy", optimizer= 'adam', metrics=["accuracy"])

In [ ]:
#------------------------------ENTRENAMIENTO----------------------------------------
pasos_por_epoca= train_labels.shape[0] // BATCH_SIZE

#Entrenamiento del modelo con los siguente parametros
#train_dataset= Data de entrenamiento
#epochs= numero de epocas que va tener el experimento , 40 epocas
#callbacks = Tasa de aprendizaje en cada epoca , TA(epoca)
#steps_per_epoch=pasos por cada epoca 

history = model.fit(
    train_dataset, 
    epochs=EPOCAS, 
    callbacks=[TA],
    steps_per_epoch=pasos_por_epoca,
    validation_data=valid_dataset if SPLIT_VALIDATION else None,
)



In [ ]:

#-------------------------------TEST----------------------------------------

#Prediccion del modelo
print("La data de pruebas contiene ",test.shape[0]," imagenes")
prediccion= model.predict(test_dataset)
print(prediccion)




In [ ]:
#-------------------------------RESULTADOS----------------------------------------


#Matriz de resultados
resultados = np.ndarray(shape = (test.shape[0],4), dtype = np.float32)

#Bucle para predecir la clase de cada imagen en la data de pruebas.
for row in range(test.shape[0]):
    for col in range(4):
        if prediccion[row][col] == max(prediccion[row]):
            resultados[row][col] = 1
        else:
            resultados[row][col] = 0
            

#rust = oxido
#scab = costra

#Mostrar Resultados del test        
resultados = pd.DataFrame(resultados)
resultados.columns = ['Sana', 'Multiple', 'oxido', 'costra']
resultados2= pd.concat([test.image_id, resultados ], axis = 1)

print(resultados2[0:100])

In [ ]:
print(test_dataset)
print(train_dataset)


In [ ]:
CATEGORY_NAMES = ['healthy','multiple_diseases','rust','scab']
nb_classes = 4

def compute_confusion_matrix(true, pred):
    result = np.zeros((nb_classes,nb_classes))
    
    true = true.argmax(axis=1)
    pred = pred.argmax(axis=1)
    
    for i in range(len(true)):
        result[true[i]][pred[i]] += 1
        
    return result.astype('uint16')

def plot_confusion_matrix(matrix, labels_x=CATEGORY_NAMES, labels_y=CATEGORY_NAMES):
    
    # vertical axis = true, horizontal axis = pred
    
    plt.title("Matrix de confusion")
    ax = sns.heatmap(matrix, annot=True, fmt='d', xticklabels = labels_x, yticklabels = labels_y)
    ax.set(ylabel="Real", xlabel="Predicho")
    
#val_pred = get_predict(model,val_dataset,USE_TTA = False)
y_pred = np.array()
y_val = np.array()
val_conf_matrix = compute_confusion_matrix(y_val,y_pred)
plot_confusion_matrix(val_conf_matrix)
